# Pancreas endocrinogenesis

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import scvelo as scv
import cellrank as cr
from cellrank.external.kernels import WOTKernel
from cellrank.estimators import GPCCA

## General settings

In [ ]:
# set verbosity levels
sc.settings.verbosity = 2
cr.settings.verbosity = 4
scv.settings.verbosity = 3

In [ ]:
scv.settings.set_figure_params('scvelo', dpi_save=400, dpi=80, transparent=True, fontsize=20, color_map='viridis')

In [ ]:
cr.logging.print_versions()

In [ ]:
PROJECT_DIR = Path("/lustre/groups/ml01/projects/2022_neural_ot/")

## Data loading

In [ ]:
adata = sc.read(PROJECT_DIR / "adata_pancreas_2019.h5ad")
adata

In [ ]:
scv.pl.scatter(adata, basis='umap', c=['day', 'celltype'])

In [ ]:
scv.pl.scatter(adata, basis='umap', c='celltype', dpi=250, legend_fontweight='normal', legend_fontsize=12)

## CellRank

### Transition probabilities

In [ ]:
wk = WOTKernel(adata, time_key="day")
wk.compute_initial_growth_rates(organism="mouse", key_added="growth_rate_init")
wk.compute_transition_matrix(
    growth_iters=3, growth_rate_key="growth_rate_init", self_transitions="all", conn_weight=0.2
)

### Macrostate calculation

In [ ]:
g = GPCCA(wk)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

__<span style="color: red;">TODO: Specify number of states</span>__

In [ ]:
g.compute_macrostates(n_states=9, cluster_key="celltype")

In [ ]:
g.plot_macrostates(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

### Fate probabilities

__<span style="color: red;">TODO: Specify terminal states</span>__

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        "Alpha", "Beta", "Delta", "Acinar_1"
    ]
)

In [ ]:
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, time_to_absorption='all')

In [ ]:
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=3)

__<span style="color: red;">TODO: Specify lineages/clusters</span>__

In [ ]:
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=['Acinar_1', 'Alpha', 'Beta', 'Delta'],  # Ductal, Epsilon
    cluster_key='celltype',
    clusters=['Multipotent', 'Tip', 'Fev+ Alpha', 'Fev+ Beta', 'Fev+ Pyy', 'Fev+ Delta', 'Fev+ Epsilon'],
    figsize=(10, 4)
)